### LIBRARIES IMPORTATION

In [1]:
import numpy as np
import pandas as pd
import os
import random
# import requests
from io import BytesIO
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, Flatten, Dropout, GlobalAveragePooling2D, Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model, load_model, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications.efficientnet import preprocess_input
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import matplotlib.image as mimg

### LOADING THE FILE AND CONVERTING 

In [2]:
# Load annotations file
train_annotations_path = 'C:/Users/ACER/Downloads/train.txt'
test_annotations_path = 'C:/Users/ACER/Downloads/test.txt'
train_image_path = 'C:/Users/ACER/Downloads/Train/Train'
test_image_path = 'C:/Users/ACER/Downloads/Test/Test'

train_data, test_data = [], []
with open(train_annotations_path, 'r') as train_file:
  train_data = train_file.read().splitlines()
with open(test_annotations_path, 'r') as test_file:
  test_data = test_file.read().splitlines()

train_df = pd.DataFrame([x.split() for x in train_data], columns = ['image_name', 'class_label'])
test_df = pd.DataFrame([x.split() for x in test_data], columns = ['image_name', 'class_label'])

In [3]:
def extract_bird_types(annotation_path):
    # Read the annotation file into a pandas DataFrame
    annotations = pd.read_csv(annotation_path, sep=' ', header=None, names=['image_name', 'class_label'])
    
    # Extract bird type from the file name and remove trailing numeric part
    annotations['bird_type'] = annotations['image_name'].apply(lambda x: '_'.join(x.split('_')[:-1]).rsplit('_', 1)[0])

    
    # Create a dictionary to associate bird types with class labels
    bird_type_to_class_label = annotations.set_index('bird_type')['class_label'].to_dict()
    
    return bird_type_to_class_label

bird_type_to_class_label = extract_bird_types(train_annotations_path)
print(bird_type_to_class_label)

{'Black_footed_Albatross': 0, 'Laysan_Albatross': 1, 'Sooty_Albatross': 2, 'Groove_billed_Ani': 3, 'Crested_Auklet': 4, 'Least_Auklet': 5, 'Parakeet_Auklet': 6, 'Rhinoceros_Auklet': 7, 'Brewer_Blackbird': 8, 'Red_winged_Blackbird': 9, 'Rusty_Blackbird': 10, 'Yellow_headed_Blackbird': 11, 'Bobolink': 12, 'Indigo_Bunting': 13, 'Lazuli_Bunting': 14, 'Painted_Bunting': 15, 'Cardinal': 16, 'Spotted_Catbird': 17, 'Gray_Catbird': 18, 'Yellow_breasted_Chat': 19, 'Eastern_Towhee': 20, 'Chuck_will_Widow': 21, 'Brandt_Cormorant': 22, 'Red_faced_Cormorant': 23, 'Pelagic_Cormorant': 24, 'Bronzed_Cowbird': 25, 'Shiny_Cowbird': 26, 'Brown_Creeper': 27, 'American_Crow': 28, 'Fish_Crow': 29, 'Black_billed_Cuckoo': 30, 'Mangrove_Cuckoo': 31, 'Yellow_billed_Cuckoo': 32, 'Gray_crowned_Rosy_Finch': 33, 'Purple_Finch': 34, 'Northern_Flicker': 35, 'Acadian_Flycatcher': 36, 'Great_Crested_Flycatcher': 37, 'Least_Flycatcher': 38, 'Olive_sided_Flycatcher': 39, 'Scissor_tailed_Flycatcher': 40, 'Vermilion_Flycatc

In [4]:
# Extract bird type from the image name in train_df
train_df['bird_type'] = train_df['image_name'].apply(lambda x: '_'.join(x.split('_')[:-1]).rsplit('_', 1)[0])

# Map the bird type to the class label using the bird_type_to_class_label dictionary
train_df['class_label'] = train_df['bird_type'].map(bird_type_to_class_label)

# Display the updated DataFrame
train_df[['image_name', 'class_label', 'bird_type']]

,image_name,class_label,bird_type
0,Black_footed_Albatross_0004_2731401028.jpg,0,Black_footed_Albatross
1,Black_footed_Albatross_0008_1384283201.jpg,0,Black_footed_Albatross
2,Black_footed_Albatross_0011_820118444.jpg,0,Black_footed_Albatross
3,Black_footed_Albatross_0003_2981373810.jpg,0,Black_footed_Albatross
4,Black_footed_Albatross_0021_2443213385.jpg,0,Black_footed_Albatross
...,...,...,...
4824,Common_Yellowthroat_0012_2573777857.jpg,199,Common_Yellowthroat
4825,Common_Yellowthroat_0002_2679007659.jpg,199,Common_Yellowthroat
4826,Common_Yellowthroat_0009_2099303008.jpg,199,Common_Yellowthroat
4827,Common_Yellowthroat_0008_2691070273.jpg,199,Common_Yellowthroat


### UNDERSTANDING THE FILE

In [5]:
import shutil

new_folder = 'C:/Users/ACER/Downloads/bird_class'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

# Define the train folder path inside the bird_class folder
train_folder = os.path.join(new_folder, 'train')

# Create the train folder if it doesn't exist
if not os.path.exists(train_folder):
    os.makedirs(train_folder)

# Create folders for each bird type inside the train folder
for bird_type in train_df['bird_type'].unique():
    bird_folder = os.path.join(train_folder, bird_type)
    if not os.path.exists(bird_folder):
        os.makedirs(bird_folder)

# Move images to the corresponding bird type folders inside the train folder
for index, row in train_df.iterrows():
    src_path = os.path.join(train_image_path, row['image_name'])
    dest_folder = os.path.join(train_folder, row['bird_type'])
    dest_path = os.path.join(dest_folder, row['image_name'])
    shutil.copy(src_path, dest_path)

In [6]:
import csv

# Convert the dictionary to a list of tuples
bird_type_list = [(key, value) for key, value in bird_type_to_class_label.items()]

# Save the list to a CSV file with two columns: "class_label" and "bird_type"
with open('bird_types.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['bird_type', 'class_label'])  # Write the header
    writer.writerows(bird_type_list)  # Write the data

In [7]:
bird_df = pd.read_csv('bird_types.csv')
bird_df

,bird_type,class_label
0,Black_footed_Albatross,0
1,Laysan_Albatross,1
2,Sooty_Albatross,2
3,Groove_billed_Ani,3
4,Crested_Auklet,4
...,...,...
195,House_Wren,195
196,Marsh_Wren,196
197,Rock_Wren,197
198,Winter_Wren,198


In [8]:
# Ensure class_label is of the same type in both dataframes
test_df['class_label'] = test_df['class_label'].astype(str)
bird_df['class_label'] = bird_df['class_label'].astype(str)

# Merge the dataframes
merged_df = pd.merge(test_df, bird_df, on='class_label', how='left')
merged_df

,image_name,class_label,bird_type
0,0000.jpg,0,Black_footed_Albatross
1,0001.jpg,0,Black_footed_Albatross
2,0002.jpg,0,Black_footed_Albatross
3,0003.jpg,0,Black_footed_Albatross
4,0004.jpg,0,Black_footed_Albatross
...,...,...,...
1199,1199.jpg,199,Common_Yellowthroat
1200,1200.jpg,199,Common_Yellowthroat
1201,1201.jpg,199,Common_Yellowthroat
1202,1202.jpg,199,Common_Yellowthroat


In [9]:
new_folder = 'C:/Users/ACER/Downloads/bird_class'
if not os.path.exists(new_folder):
    os.makedirs(new_folder)

# Define the train folder path inside the bird_class folder
test_folder = os.path.join(new_folder, 'test')

# Create the train folder if it doesn't exist
if not os.path.exists(test_folder):
    os.makedirs(test_folder)

# Create folders for each bird type inside the train folder
for bird_type in merged_df['bird_type'].unique():
    bird_folder = os.path.join(test_folder, bird_type)
    if not os.path.exists(bird_folder):
        os.makedirs(bird_folder)

# Move images to the corresponding bird type folders inside the train folder
for index, row in merged_df.iterrows():
    src_path = os.path.join(test_image_path, row['image_name'])
    dest_folder = os.path.join(test_folder, row['bird_type'])
    dest_path = os.path.join(dest_folder, row['image_name'])
    shutil.copy(src_path, dest_path)

### ImageDataGenerator

In [10]:
train_gen=ImageDataGenerator(rescale=1./255, rotation_range = 40,
                                   width_shift_range = 0.2,
                                   height_shift_range = 0.2,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
train_set=train_gen.flow_from_directory('C:/Users/ACER/Downloads/bird_class/train',target_size=(224,224),class_mode='categorical',batch_size=32)

test_gen=ImageDataGenerator(rescale=1./255)
test_set=test_gen.flow_from_directory('C:/Users/ACER/Downloads/bird_class/test',target_size=(224,224),class_mode='categorical',batch_size=32)

Found 4829 images belonging to 200 classes.
Found 1204 images belonging to 200 classes.


In [11]:
train_gen = ImageDataGenerator(rescale=1./255, rotation_range=40, width_shift_range=0.2,
                               height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                               horizontal_flip=True)
train_set = train_gen.flow_from_directory('C:/Users/ACER/Downloads/bird_class/train',
                                          target_size=(224, 224), class_mode='categorical', batch_size=32)

test_gen = ImageDataGenerator(rescale=1./255)
test_set = test_gen.flow_from_directory('C:/Users/ACER/Downloads/bird_class/test',
                                        target_size=(224, 224), class_mode='categorical', batch_size=32)


Found 4829 images belonging to 200 classes.
Found 1204 images belonging to 200 classes.


In [12]:
def evaluate_model(model, test_generator):
    test_generator.reset()
    preds = model.predict(test_generator)
    y_true = test_generator.classes
    y_pred = np.argmax(preds, axis=1)

    # Top-1 Accuracy
    top1_accuracy = accuracy_score(y_true, y_pred)

    # Average Accuracy per class
    report = classification_report(y_true, y_pred, output_dict=True)
    avg_accuracy_per_class = np.mean([report[str(i)]['recall'] for i in range(200)])  # For 200 classes

    return top1_accuracy, avg_accuracy_per_class

In [13]:
# Check if the generators are working
print(f"Number of training samples: {train_set.samples}")
print(f"Number of testing samples: {test_set.samples}")

# Ensure the test generator is not empty
if test_set.samples == 0:
    raise ValueError("Test generator returned no samples. Please check the test directory and the path.")


Number of training samples: 4829
Number of testing samples: 1204


### SIMPLE CNN MODEl

In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, classification_report

# Data generators for loading images
datagen = ImageDataGenerator(rescale=1./255)

# Specify the correct directory paths for train and test sets
train_dir = 'C:/Users/ACER/Downloads/bird_class/train'
test_dir = 'C:/Users/ACER/Downloads/bird_class/test'

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # Image size (you can adjust if needed)
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False  # No shuffling for test data
)

# CNN Model Definition
cnn_model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D(2, 2),
    layers.Flatten(),
    layers.Dense(512, activation='relu'),
    layers.Dense(200, activation='softmax')  # Assuming 200 bird classes
])

# Compile the model
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the CNN model
cnn_model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the CNN model
test_loss, test_accuracy = cnn_model.evaluate(test_generator)
print(f"Test Accuracy (CNN): {test_accuracy}")

# Getting predictions and calculating Top-1 Accuracy
test_generator.reset()  # Reset the generator before making predictions
preds = cnn_model.predict(test_generator, verbose=1)

# Get true labels and predicted labels
y_true = test_generator.classes
y_pred = preds.argmax(axis=1)

# Calculate accuracy score and classification report
accuracy = accuracy_score(y_true, y_pred)
print(f"Top-1 Accuracy: {accuracy}")

# Detailed classification report
report = classification_report(y_true, y_pred, target_names=list(test_generator.class_indices.keys()))
print(report)


Found 4829 images belonging to 200 classes.
Found 1204 images belonging to 200 classes.


c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 235s 2s/step - accuracy: 0.0042 - loss: 5.3962 - val_accuracy: 0.0050 - val_loss: 5.3100
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 199s 1s/step - accuracy: 0.0104 - loss: 5.2913 - val_accuracy: 0.0174 - val_loss: 5.2129
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.0971 - loss: 4.7278 - val_accuracy: 0.0266 - val_loss: 5.4092
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 204s 1s/step - accuracy: 0.5271 - loss: 2.4035 - val_accuracy: 0.0257 - val_loss: 8.7995
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 195s 1s/step - accuracy: 0.8669 - loss: 0.7752 - val_accuracy: 0.0307 - val_loss: 12.6471
Epoch 6/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 236s 2s/step - accuracy: 0.9555 - loss: 0.4159 - val_accuracy: 0.0299 - val_loss: 14.9106
Epoch 7/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 215s 1s/step - accuracy: 0.9809 - loss: 0.1846 - val_accuracy: 0.0291 - val_loss: 14.5671
Epoch 8/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 226s 1s/step - accuracy: 0.9906 - loss: 0.0556 - val_a

c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [15]:
import numpy as np

# Calculate Top-5 Accuracy
top_5_preds = np.argsort(preds, axis=1)[:, -5:]  # Get the indices of top 5 predictions
top_5_correct = np.any(top_5_preds == y_true[:, None], axis=1)  # Check if true label is in top 5
top_5_accuracy = np.mean(top_5_correct)

print(f"Top-5 Accuracy: {top_5_accuracy}")


Top-5 Accuracy: 0.0946843853820598


In [16]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Getting true labels and predicted labels
y_true = test_generator.classes
y_pred = preds.argmax(axis=1)

# Top-1 Accuracy
top1_accuracy_cnn = accuracy_score(y_true, y_pred)
print(f"Top-1 Accuracy (CNN): {top1_accuracy_cnn}")

# Generate classification report with zero_division to handle undefined metrics
report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

# Collect recalls for all classes that are present in the report
recalls = []
for i in range(200):  # Assuming 200 classes
    class_label = str(i)
    if class_label in report:  # Check if the class exists in the report
        recalls.append(report[class_label]['recall'])

# Calculate Average Accuracy per Class
if recalls:
    avg_accuracy_per_class = np.mean(recalls)
    print(f"Average Accuracy per Class (CNN): {avg_accuracy_per_class}")
else:
    print("No recalls available for calculation.")


Top-1 Accuracy (CNN): 0.03322259136212625
Average Accuracy per Class (CNN): 0.030809523809523807


In [17]:
# Save the trained model to a .h5 file
cnn_model.save('C:/Users/ACER/Downloads/cnn_model.h5')

# To load the model later
loaded_model = tf.keras.models.load_model('C:/Users/ACER/Downloads/cnn_model.h5')

### RESNET MODEL

In [18]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers, models
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Set the paths to your training and testing directories
train_dir = 'C:/Users/ACER/Downloads/bird_class/train'
test_dir = 'C:/Users/ACER/Downloads/bird_class/test'

# Data generators for loading images (assuming 224x224 image size)
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),  # ResNet50 expects 224x224 images
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Load the pre-trained ResNet50 model without the top layer
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of ResNet50 to prevent them from being updated during training
resnet_base.trainable = False

# Build the complete model by adding custom layers on top of ResNet50
resnet_model = models.Sequential([
    resnet_base,
    layers.GlobalAveragePooling2D(),  # Pool the features from ResNet50
    layers.Dense(512, activation='relu'),
    layers.Dense(200, activation='softmax')  # Assuming 200 bird classes
])

# Compile the model
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
resnet_model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
test_loss, test_accuracy = resnet_model.evaluate(test_generator)
print(f"Test Accuracy (ResNet50): {test_accuracy}")

# Get predictions and calculate Top-1 Accuracy
test_generator.reset()
preds = resnet_model.predict(test_generator, verbose=1)

# Get true labels and predicted labels
y_true = test_generator.classes
y_pred = preds.argmax(axis=1)

# Print classification report
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))


Found 4829 images belonging to 200 classes.
Found 1204 images belonging to 200 classes.
Epoch 1/10


c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


151/151 ━━━━━━━━━━━━━━━━━━━━ 363s 2s/step - accuracy: 0.0073 - loss: 5.4021 - val_accuracy: 0.0058 - val_loss: 5.2930
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 351s 2s/step - accuracy: 0.0048 - loss: 5.2896 - val_accuracy: 0.0050 - val_loss: 5.2709
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 346s 2s/step - accuracy: 0.0060 - loss: 5.2635 - val_accuracy: 0.0058 - val_loss: 5.2595
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 322s 2s/step - accuracy: 0.0124 - loss: 5.2426 - val_accuracy: 0.0141 - val_loss: 5.2526
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 320s 2s/step - accuracy: 0.0102 - loss: 5.2378 - val_accuracy: 0.0066 - val_loss: 5.2524
Epoch 6/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - accuracy: 0.0131 - loss: 5.2200 - val_accuracy: 0.0116 - val_loss: 5.2446
Epoch 7/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - accuracy: 0.0165 - loss: 5.2012 - val_accuracy: 0.0108 - val_loss: 5.2404
Epoch 8/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 318s 2s/step - accuracy: 0.0129 - loss: 5.1905 - val_accuracy: 0.013

c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

# Top-1 Accuracy
top1_accuracy_resnet = accuracy_score(y_true, y_pred)
print(f"Top-1 Accuracy (ResNet50): {top1_accuracy_resnet}")

# Generate classification report with zero_division=0 to avoid issues with undefined metrics
report = classification_report(y_true, y_pred, output_dict=True, zero_division=0)

# Collect recalls for all classes that exist in the report
recalls = []
for i in range(200):  # Assuming 200 classes
    class_label = str(i)
    if class_label in report:  # Check if the class exists in the report
        recalls.append(report[class_label]['recall'])

# Calculate Average Accuracy per Class
if recalls:
    avg_accuracy_per_class = np.mean(recalls)
    print(f"Average Accuracy per Class (ResNet50): {avg_accuracy_per_class}")
else:
    print("No recalls available for calculation.")


resnet_model.save('C:/Users/ACER/Downloads/resnet50_model.h5')



Top-1 Accuracy (ResNet50): 0.012458471760797342
Average Accuracy per Class (ResNet50): 0.01050595238095238


### Inception V3

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
import numpy as np

# Set the paths to your training and testing directories
train_dir = 'C:/Users/ACER/Downloads/bird_class/train'
test_dir = 'C:/Users/ACER/Downloads/bird_class/test'

# Data generators for loading images (assuming 299x299 image size for InceptionV3)
datagen = ImageDataGenerator(rescale=1./255)

train_generator = datagen.flow_from_directory(
    train_dir,
    target_size=(299, 299),  # InceptionV3 expects 299x299 images
    batch_size=32,
    class_mode='categorical'
)

test_generator = datagen.flow_from_directory(
    test_dir,
    target_size=(299, 299),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Load the pre-trained InceptionV3 model without the top layer
inception_base = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))

# Freeze the layers of InceptionV3 to prevent them from being updated during training
inception_base.trainable = False

# Build the complete model by adding custom layers on top of InceptionV3
inception_model = models.Sequential([
    inception_base,
    layers.GlobalAveragePooling2D(),  # Pool the features from InceptionV3
    layers.Dense(512, activation='relu'),
    layers.Dense(200, activation='softmax')  # Assuming 200 bird classes
])

# Compile the model
inception_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
inception_model.fit(train_generator, epochs=10, validation_data=test_generator)

# Evaluate the model
test_loss, test_accuracy = inception_model.evaluate(test_generator)
print(f"Test Accuracy (InceptionV3): {test_accuracy}")

# Get predictions and calculate Top-1 Accuracy
test_generator.reset()
preds = inception_model.predict(test_generator, verbose=1)

# Get true labels and predicted labels
y_true = test_generator.classes
y_pred = preds.argmax(axis=1)

# Print classification report
print(classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))


Found 4829 images belonging to 200 classes.
Found 1204 images belonging to 200 classes.


c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 379s 2s/step - accuracy: 0.0587 - loss: 4.9412 - val_accuracy: 0.2126 - val_loss: 3.3698
Epoch 2/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 319s 2s/step - accuracy: 0.3124 - loss: 2.8314 - val_accuracy: 0.3239 - val_loss: 2.6431
Epoch 3/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 315s 2s/step - accuracy: 0.4738 - loss: 2.0530 - val_accuracy: 0.4128 - val_loss: 2.2775
Epoch 4/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 310s 2s/step - accuracy: 0.5605 - loss: 1.6315 - val_accuracy: 0.4269 - val_loss: 2.2253
Epoch 5/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 311s 2s/step - accuracy: 0.6354 - loss: 1.3346 - val_accuracy: 0.4767 - val_loss: 2.0831
Epoch 6/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 311s 2s/step - accuracy: 0.7047 - loss: 1.0909 - val_accuracy: 0.4842 - val_loss: 1.9864
Epoch 7/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 310s 2s/step - accuracy: 0.7750 - loss: 0.8635 - val_accuracy: 0.5233 - val_loss: 1.8931
Epoch 8/10
151/151 ━━━━━━━━━━━━━━━━━━━━ 311s 2s/step - accuracy: 0.8094 - loss: 0.7296 - val_accu

c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [22]:
# Top-1 Accuracy
y_true = test_generator.classes
y_pred = preds.argmax(axis=1)
print(f"Top-1 Accuracy: {accuracy_score(y_true, y_pred)}")
# Calculate Average Accuracy per Class
avg_accuracy_per_class = np.mean([report[label]['precision'] for label in report if label != 'accuracy'])
print(f"Average Accuracy per Class: {avg_accuracy_per_class:.4f}")


Top-1 Accuracy: 0.5290697674418605
Average Accuracy per Class: 0.0028


In [23]:
# Classification Report (Average accuracy per class)
print("Classification Report:\n", classification_report(y_true, y_pred, target_names=test_generator.class_indices.keys()))

# Confusion Matrix
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))


c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\ACER\anaconda3\envs\myenv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Classification Report:
                                 precision    recall  f1-score   support

            Acadian_Flycatcher       0.22      0.71      0.33         7
                 American_Crow       0.22      0.40      0.29         5
            American_Goldfinch       0.50      0.80      0.62         5
                American_Pipit       1.00      0.33      0.50         6
             American_Redstart       1.00      0.33      0.50         6
American_Three_toed_Woodpecker       0.50      0.50      0.50         6
              Anna_Hummingbird       0.50      0.60      0.55         5
                    Artic_Tern       0.33      0.14      0.20         7
                 Baird_Sparrow       0.19      0.43      0.26         7
              Baltimore_Oriole       0.33      0.40      0.36         5
                  Bank_Swallow       0.00      0.00      0.00         5
                  Barn_Swallow       0.30      0.60      0.40         5
          Bay_breasted_Warbler       0.

NameError: name 'confusion_matrix' is not defined

In [24]:
# Top-5 Accuracy
top_5_preds = np.argsort(preds, axis=1)[:, -5:]
top_5_acc = np.mean([y_true[i] in top_5_preds[i] for i in range(len(y_true))])
print(f"Top-5 Accuracy: {top_5_acc}")

Top-5 Accuracy: 0.8272425249169435
